In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
# Getting file from local, change file path necessary
master = pd.read_csv("/Users/ziqiancui/Desktop/Gloabl AI/Yahoo Finance/master_updated.csv")
master_ticker = master['Ticker'].tolist()
#mystocks = ['GME', 'RKT', 'TSLA', 'AMC', 'BB', 'NOK']
stockdata = []

def getData(symbol):
    # Get User-Agent, change needed for each browser
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    
    # Yahoo Finance URL
    url = f'https://finance.yahoo.com/quote/{symbol}'

    # Stcok Profile URL
    url_profile = f'https://finance.yahoo.com/quote/{symbol}/profile?p={symbol}'
    
    # Request URL
    r = requests.get(url, headers=headers)
    r_profile = requests.get(url_profile, headers=headers)

    # Create BeautifulSoup object
    soup = BeautifulSoup(r.text, 'html.parser')
    soup_profile = BeautifulSoup(r_profile.text, 'html.parser')

    # Check if stock exists, if not return None
    if soup.find('div', {'id': 'quote-header-info'}) is None:
        return {'symbol': symbol,
                'price': None,
                'market cap': None,
                'exchange': None,
                'currency': None,
                'sector': None,
                'industry': None,
                'company_url': None}
    
    # Exchange pattern for exchange and currency
    exchange_pattern = re.compile(r'^.*(?=\s-\s)')
    currency_pattern = re.compile(r'(?<=in\s).*')

    # Find div element containing currency info
    currency_div = soup.find('div', {'class': 'C($tertiaryColor) Fz(12px)'}) if soup.find('div', {'class': 'C($tertiaryColor) Fz(12px)'}) else None

    # Get stock data
    stock = {
    'symbol': symbol,
    'price': soup.find('div', {'class': 'D(ib) Mend(20px)'}).find_all('fin-streamer')[0].text if soup.find('div', {'class': 'D(ib) Mend(20px)'}) else None, 
    'market cap': soup.find('tr', {'class': 'Bxz(bb) Bdbw(1px) Bdbs(s) Bdc($seperatorColor) H(36px)'}).find_all('td')[1].text if soup.find('tr', {'class': 'Bxz(bb) Bdbw(1px) Bdbs(s) Bdc($seperatorColor) H(36px)'}) else None,
    'exchange': re.search(exchange_pattern, currency_div.text).group() if currency_div and re.search(exchange_pattern, currency_div.text) else None,
    'currency': re.search(currency_pattern, currency_div.text).group() if currency_div and re.search(currency_pattern, currency_div.text) else None,
    'sector': soup_profile.find("p", {'class': "D(ib) Va(t)"}).find_all('span')[1].text if soup_profile.find("p", {'class': "D(ib) Va(t)"}) else None,
    'industry': soup_profile.find("p", {'class': "D(ib) Va(t)"}).find_all('span')[3].text if soup_profile.find("p", {'class': "D(ib) Va(t)"}) else None,
    'company_url': soup_profile.find('p', {'class': 'D(ib) W(47.727%) Pend(40px)'}).find_all('a')[1].get('href') if soup_profile.find('p', {'class': 'D(ib) W(47.727%) Pend(40px)'}) else None
    }

    return stock

# Get stock data for each stock in mystocks
for item in master_ticker:
    try:
        stock = getData(item)
        if stock is not None:
            stockdata.append(stock)
            print("getting: ", item)
    except requests.exceptions.RequestException as e:
        print(f"Error getting data for {item}: {e}")
        continue

# Create DataFrame from stockdata
df = pd.DataFrame.from_dict(stockdata)

# Drop rows with missing values
df = df.dropna(subset=['price'])
df = df.dropna()
df = df.drop('Unnamed: 0', axis=1)

# To CSV
df.to_csv("Yahoo_Finance.csv")